In [ ]:
# import necessary modules and display matplotlib plots inline within the ipython notebook webpage

import pandas as pd, numpy as np, statsmodels.api as sm
import matplotlib.pyplot as plt, matplotlib.cm as cm, matplotlib.font_manager as fm
import matplotlib.mlab as mlab
from scipy.stats import pearsonr, ttest_rel
%matplotlib inline

In [ ]:
with open('pg_engine.txt') as f:
    pg_engine = f.readlines()
from sqlalchemy import create_engine
engine = create_engine(pg_engine[0])

In [ ]:
import pandas as pd
df = pd.read_sql_query('select * from "rental_listings"',con=engine)

In [ ]:
print(df.dtypes)
df.describe()


In [ ]:
# convert the date column to yyyy-mm-dd date format
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

In [ ]:
def get_colors(cmap, n, start=0., stop=1., alpha=1., reverse=False):
    '''return n-length list of rgba colors from the passed colormap name and alpha,
       limit extent by start/stop values and reverse list order if flag is true'''
    colors = [cm.get_cmap(cmap)(x) for x in np.linspace(start, stop, n)]
    colors = [(r, g, b, alpha) for r, g, b, _ in colors]
    return list(reversed(colors)) if reverse else colors

In [ ]:
# define the font styles
family = 'Arial'
title_font = fm.FontProperties(family=family, style='normal', size=18, weight='normal', stretch='normal')
label_font = fm.FontProperties(family=family, style='normal', size=16, weight='normal', stretch='normal')
ticks_font = fm.FontProperties(family=family, style='normal', size=14, weight='normal', stretch='normal')

In [ ]:
# function to save images consistently
save_dpi = [96, 300]
def save_fig(fig, title, tight=True):    
    if tight:
        fig.tight_layout()
    for dpi in save_dpi:
        save_folder = 'images/dpi_{}/'.format(dpi)
        fig.savefig(save_folder + title, dpi=dpi)

In [ ]:
# create ticks and tick labels for the time series
listings_per_date = df['date'].value_counts()
listings_per_date = listings_per_date.sort_index()
listings_per_date = listings_per_date.reset_index()
xticks = listings_per_date.iloc[range(0, len(listings_per_date), 7)].index
xtick_labels = listings_per_date.loc[xticks, 'index']
xtick_labels = [str(x).split()[0] for x in xtick_labels]

In [ ]:
# plot the total number of listings (includes dupes/re-posts) posted on each day in the data set
ax = listings_per_date.plot(kind='line', figsize=[10, 6], ylim=[0,100000], linewidth=3, 
                            marker='o', markeredgewidth=0, alpha=0.7, color='#003399')
ax.grid(True)
ax.set_title('Total rental listings posted per day', fontproperties=title_font)
ax.set_ylabel('Number of listings posted', fontproperties=label_font)
ax.legend_.remove()

ax.set_xticks(xticks)
ax.set_xticklabels(xtick_labels, rotation=40, rotation_mode='anchor', ha='right', fontproperties=ticks_font)
for label in ax.get_yticklabels():
    label.set_fontproperties(ticks_font)

save_fig(plt.gcf(), 'date_count_listings_posted.png')
plt.show()

In [ ]:
#store = pd.HDFStore('data/rents.h5')
#store['rents'] = df
df.to_hdf('data/rents.h5','rents',append=False)

In [ ]:
store = pd.HDFStore('data/rents.h5')
store

In [ ]:
upper_percentile = 0.998
lower_percentile = 0.002

# how many rows would be within the upper and lower percentiles?
upper = int(len(df) * upper_percentile)
lower = int(len(df) * lower_percentile)

# get the rent/sqft values at the upper and lower percentiles
rent_sqft_sorted = df['rent_sqft'].sort_values(ascending=True, inplace=False)
upper_rent_sqft = rent_sqft_sorted.iloc[upper]
lower_rent_sqft = rent_sqft_sorted.iloc[lower]

# get the rent values at the upper and lower percentiles
rent_sorted = df['rent'].sort_values(ascending=True, inplace=False)
upper_rent = rent_sorted.iloc[upper]
lower_rent = rent_sorted.iloc[lower]

# get the sqft values at the upper and lower percentiles
sqft_sorted = df['sqft'].sort_values(ascending=True, inplace=False)
upper_sqft = sqft_sorted.iloc[upper]
lower_sqft = sqft_sorted.iloc[lower]

print('valid rent_sqft range:', [lower_rent_sqft, upper_rent_sqft])
print('valid rent range:', [lower_rent, upper_rent])
print('valid sqft range:', [lower_sqft, upper_sqft])

In [ ]:
# create a boolean vector mask to filter out any rows with rent_sqft outside of the reasonable values
rent_sqft_mask = (df['rent_sqft'] > lower_rent_sqft) & (df['rent_sqft'] < upper_rent_sqft)

# create boolean vector masks to filter out any rows with rent or sqft outside of the reasonable values
rent_mask = (df['rent'] > lower_rent) & (df['rent'] < upper_rent)
sqft_mask = (df['sqft'] > lower_sqft) & (df['sqft'] < upper_sqft)

# filter the thorough listings according to these masks
filtered_listings = pd.DataFrame(df[rent_sqft_mask & rent_mask & sqft_mask])
len(filtered_listings)

In [ ]:
filtered_listings.describe()

In [ ]:
sfbay = filtered_listings[filtered_listings['region']=='sfbay']
sfbay.describe()

In [ ]:
sfbay['rent_sqft'].quantile(.01)

In [ ]:
sfbay['sqft'].quantile(.01)

In [ ]:
# create a boolean vector mask to filter out any rows with rent_sqft and sqft in Bay Area under 1 percentile
sfbay_rent_sqft_mask = (sfbay['rent_sqft'] > sfbay['rent_sqft'].quantile(.01) )

# create boolean vector masks to filter out any rows with rent or sqft outside of the reasonable values
sfbay_sqft_mask = (sfbay['sqft'] > sfbay['sqft'].quantile(.01) )

# filter the thorough listings according to these masks
sfbay_filtered = pd.DataFrame(sfbay[sfbay_rent_sqft_mask & sfbay_sqft_mask])
len(sfbay_filtered)

In [ ]:
sfbay_filtered.describe()

In [ ]:
geocoded = pd.read_csv('data/craigslist_data_wblockid.csv', dtype={'GEOID10': object}).rename(columns={'GEOID10':'fips_block'})

In [ ]:
print(geocoded.columns)
geocoded.describe()

In [ ]:
geocoded[:10]

In [ ]:
sfbay_geocoded = geocoded[geocoded['region']=='sfbay']
sfbay_geocoded.describe()

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf
import pandas as pd

US_XMin = -124.848974
US_XMax = -66.885444

US_YMin = 24.396308
US_YMax = 49.384358

geocoded = geocoded[(geocoded['longitude']>US_XMin) & (geocoded['longitude']<US_XMax)]
geocoded = geocoded[(geocoded['latitude']>US_YMin) & (geocoded['latitude']<US_YMax)]

cvs = ds.Canvas(plot_width=1000, plot_height=600)
agg = cvs.points(geocoded, 'longitude', 'latitude', ds.mean('rent_sqft'))
img = tf.shade(agg, cmap=['lightgreen', 'darkblue'], how='log')
img

In [ ]:
filtered_listings = filtered_listings[(filtered_listings['longitude']>US_XMin) & (filtered_listings['longitude']<US_XMax)]
filtered_listings = filtered_listings[(filtered_listings['latitude']>US_YMin) & (filtered_listings['latitude']<US_YMax)]

cvs = ds.Canvas(plot_width=1000, plot_height=600)
agg = cvs.points(filtered_listings, 'longitude', 'latitude', ds.mean('rent_sqft'))
img = tf.shade(agg, cmap=['lightgreen', 'darkblue'], how='log')
img

In [ ]:
import datashader as ds 
import datashader.transfer_functions as tf
from datashader.colors import Greys9, Hot, colormap_select as cm 
def bg(img): return tf.set_background(img,"black")

In [ ]:
USA = ((-124.848974, -66.885444), (24.396308, 49.384358))
 
x_range,y_range = USA
 
plot_width = int(900)
plot_height = int(plot_width*7.0/12)

In [ ]:
def create_image(x_range, y_range, w=plot_width, h=plot_height, spread=0):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.points(df, 'meterswest', 'metersnorth', ds.count_cat('race'))
    img = tf.colorize(agg, color_key, how='eq_hist')
    if spread: img = tf.spread(img,px=spread)
    return tf.set_background(img,"black")

In [ ]:
import bokeh.plotting as bp
from bokeh.models.tiles import WMTSTileSource
 
bp.output_notebook()
 
def base_plot(tools='pan,wheel_zoom,reset',webgl=False):
     p = bp.figure(tools=tools,
         plot_width=int(900), plot_height=int(500),
         x_range=x_range, y_range=y_range, outline_line_color=None,
         min_border=0, min_border_left=0, min_border_right=0,
         min_border_top=0, min_border_bottom=0, webgl=webgl)
 
     p.axis.visible = False
     p.xgrid.grid_line_color = None
     p.ygrid.grid_line_color = None
 
     return p
 
p = base_plot()
 
url="http://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.png" 
tile_renderer = p.addtile(WMTSTileSource(url=url)) 
tile_renderer.alpha=1.0

In [ ]:
import statsmodels.api as sm
import numpy as np
from patsy import dmatrices
y, X = dmatrices('np.log(rent) ~ np.log(sqft) + bedrooms', data=sfbay_geocoded, return_type='dataframe')
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

In [ ]:
errors =  res.resid
errors.hist(bins=25)

In [ ]:
pred = res.fittedvalues
pred.hist(bins=25)

In [ ]:
plt.figure(1, figsize=(10,8), )
plt.scatter(res.fittedvalues, res.resid, s=.03, color='green')
plt.show();

In [ ]:
import statsmodels.api as sm
import numpy as np
from patsy import dmatrices
y, X = dmatrices('np.log(rent) ~ np.log(sqft) + bedrooms', data=sfbay_filtered, return_type='dataframe')
mod = sm.OLS(y, X)
res = mod.fit()
residuals = res.resid
predicted = res.fittedvalues
observed = y
print(res.summary())

In [ ]:

plt.hist(residuals, bins=25, normed=True, alpha=.5)
mu = residuals.mean()
variance = residuals.var()
sigma = residuals.std()
x = np.linspace(-3, 3, 100)
plt.plot(x,mlab.normpdf(x, mu, sigma));

In [ ]:
plt.figure(1, figsize=(10,8), )
plt.plot([7, 9], [0, 0], c='b')
plt.scatter(predicted, residuals, marker=0, s=2, c='g');
plt.axis([7.25, 9, -1.5, 1.5])
plt.show();

In [ ]:
plt.figure(1, figsize=(10,8), )
plt.plot([6, 9.5], [6, 9.5])
plt.scatter(observed, predicted, marker=0, s=2, c='g')
plt.axis([6.5, 9.5, 6.5, 9.5])
plt.show();

In [ ]:
print(residuals.mean())
print(residuals.std())

In [ ]:
mod = sm.WLS(y, X, weights=1.)
res = mod.fit()
print(res.summary())

In [ ]:
from pymc3 import Model, NUTS, sample
from pymc3.glm import glm

with Model() as model_glm:
    glm('np.log(rent) ~ np.log(sqft) + bedrooms + bathrooms', sfbay_filtered)
    trace = sample(5000)

In [ ]:
from pymc3 import traceplot
%matplotlib inline
traceplot(trace);

In [ ]:
from scipy import optimize
from pymc3 import find_MAP
with model_glm:

    # obtain starting values via MAP
    start = find_MAP(fmin=optimize.fmin_powell)

    # draw 2000 posterior samples
    trace = sample(5000, start=start)

In [ ]:
traceplot(trace);

In [ ]:
import matplotlib.pyplot as plt
import theano
import pymc3 as pm

In [ ]:
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, xlabel='x', ylabel='y', title='Generated data and underlying model')
ax.plot(np.log(sfbay['sqft']), np.log(sfbay['rent']), 'o', markersize=.5, color='blue', label='sampled data')
#ax.plot(x, true_regression_line, label='true regression line', lw=2.)
#pm.glm.plot_posterior_predictive(trace, samples=100,
#                                 label='posterior predictive regression lines')
plt.legend(loc=0);

In [ ]:
#Write out selected counties for student use
sanfrancisco = geocoded[geocoded['fips_block'].str.startswith('06075')]
alameda = geocoded[geocoded['fips_block'].str.startswith('06001')]
denver = geocoded[geocoded['fips_block'].str.startswith('08031')]
washingtondc = geocoded[geocoded['fips_block'].str.startswith('11001')]
king = geocoded[geocoded['fips_block'].str.startswith('53033')]
cook = geocoded[geocoded['fips_block'].str.startswith('17031')]
neworleans = geocoded[geocoded['fips_block'].str.startswith('22071')]
suffolk = geocoded[geocoded['fips_block'].str.startswith('25025')]
manhattan = geocoded[geocoded['fips_block'].str.startswith('36061')]
kings = geocoded[geocoded['fips_block'].str.startswith('36047')]
staten = geocoded[geocoded['fips_block'].str.startswith('36085')]
bronx = geocoded[geocoded['fips_block'].str.startswith('36005')]
queens = geocoded[geocoded['fips_block'].str.startswith('36081')]
wayne = geocoded[geocoded['fips_block'].str.startswith('42127')]
multnomah = geocoded[geocoded['fips_block'].str.startswith('41051')]
austin = geocoded[geocoded['fips_block'].str.startswith('48015')]

sanfrancisco.to_csv('sanfrancisco.csv')
alameda.to_csv('alameda.csv')
denver.to_csv('denver.csv')
washingtondc.to_csv('washingtondc.csv')
king.to_csv('king.csv')
cook.to_csv('cook.csv')
neworleans.to_csv('neworleans.csv')
suffolk.to_csv('suffolk.csv')
manhattan.to_csv('manhattan.csv')
kings.to_csv('kings.csv')
staten.to_csv('staten.csv')
bronx.to_csv('bronx.csv')
queens.to_csv('queens.csv')
wayne.to_csv('wayne.csv')
multnomah.to_csv('multnomah.csv')
austin.to_csv('austin.csv')



In [ ]:
austin = geocoded[geocoded['fips_block'].str.startswith('48453')]
austin.describe()
austin.to_csv('austin.csv')

In [ ]:
middlesex = geocoded[geocoded['fips_block'].str.startswith('25017')]
middlesex.describe()
#middlesex.to_csv('middlesex.csv')
